In [169]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from urllib.parse import urljoin

In [170]:
def setup_driver(headless=False):
    options = webdriver.ChromeOptions()

    # Run without opening browser window if needed
    if headless:
        options.add_argument("--headless")

    # Optional: use a Chrome profile (commented out here)
    # options.add_argument("user-data-dir=/path/to/profile")

    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [171]:
driver = setup_driver(headless=False)
wait = WebDriverWait(driver, 15)

# Dummy URL — original domain removed for privacy
base_url = "https://example-website.com"
start_url = f"{base_url}/directory/search"

driver.get(start_url)

In [174]:
driver = setup_driver(headless=False)
wait = WebDriverWait(driver, 15)

# Dummy URL — original domain removed for privacy
base_url = "https://example-website.com"
start_url = f"{base_url}/directory/search"

driver.get(start_url)

In [175]:
search_button = wait.until(
    EC.element_to_be_clickable((By.ID, "searchButton"))
)
search_button.click()

# Wait for result entries to load
entries = wait.until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.results > li"))
)

print("Number of entries found:", len(entries))

Bulunan entry sayısı: 50


In [176]:
company_links = []

for item in entries:
    a_tag = item.find_element(By.TAG_NAME, "a")
    company_name = a_tag.text.strip()
    detail_url = a_tag.get_attribute("href")

    full_url = urljoin(base_url, detail_url)

    company_links.append({
        "company": company_name,
        "detail_url": full_url
    })

company_links[:5]  # show sample output

In [ ]:
records = []
allowed_domain = "example-website.com"

for link in company_links:
    detail_url = link["detail_url"]

    # Skip URLs from outside the allowed domain (safety check)
    if allowed_domain not in detail_url:
        continue

    try:
        driver.get(detail_url)
        time.sleep(1)

        # These CSS selectors are placeholders
        # Real project selectors removed for confidentiality
        name = driver.find_element(By.CSS_SELECTOR, ".company-name").text.strip()
        city = driver.find_element(By.CSS_SELECTOR, ".location-city").text.strip()
        years_active = driver.find_element(By.CSS_SELECTOR, ".years-active").text.strip()

        records.append({
            "company_name": name,
            "city": city,
            "years_active": years_active,
            "detail_url": detail_url
        })

    except Exception as e:
        print("Failed to read page:", detail_url)
        continue

In [179]:
df = pd.DataFrame(records)
df.to_csv("sample_output_data.csv", index=False)

df.head()